# $\alpha$=0.1

In [9]:
import torch
import numpy as np
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR10      
from torch.utils.data import DataLoader
from src.temperature_scaling import ModelWithTemperature
from src.raps import raps_test
from src.inception import inception_v3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dict_path = "C:\\Users\\jiayang\\ipynb\\trainedModel\\Inception_CIFAR10.pth"
model = inception_v3(pretrained=True, dict_path=dict_path).to(device)

# preprocess the images from CIFAR10
data_transform = transforms.Compose([
    transforms.ToTensor(),       
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  
])

# load images from CIFAR10
dataset = CIFAR10(root="../../data", train=False, download=True, transform=data_transform)

# Temperature Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=5.2).to(device)
model.set_temperature(temp_scal_loader)
model.eval()

raps_test(model, dataset, device, num_runs=10, alpha=0.1, lambda_=0.2, k_reg=2)

Loading weights from: C:\Users\jiayang\ipynb\trainedModel\Inception_CIFAR10.pth
Files already downloaded and verified
Before temperature - NLL: 0.416, ECE: 0.053
Optimal temperature: 5.128
After temperature - NLL: 0.766, ECE: 0.340
RAPS Classification, Start!

Running experiment 1/10...
q_hat = 0.6562867224216462
Total set size: 7498
Total coverage sets: 4476
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.4996
Average Coverage Rate in runs 1: 0.8952

Running experiment 2/10...
q_hat = 0.6541639685630799
Total set size: 7539
Total coverage sets: 4463
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.5078
Average Coverage Rate in runs 2: 0.8926

Running experiment 3/10...
q_hat = 0.652672201395035
Total set size: 7521
Total coverage sets: 4467
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.5042
Average Coverage Rate in runs 3: 0.8934

Running experiment 4/10...
q_hat = 0.6568251013755799
Total set s

# $\alpha$=0.05

In [1]:
raps_test(model, dataset, device, num_runs=10, alpha=0.05, lambda_=0.2, k_reg=2)

Loading weights from: C:\Users\jiayang\ipynb\trainedModel\Inception_CIFAR10.pth
Files already downloaded and verified
Before temperature - NLL: 0.368, ECE: 0.030
Optimal temperature: 5.125
After temperature - NLL: 0.892, ECE: 0.409
RAPS Classification, Start!

Running experiment 1/10...
q_hat = 0.7119233638048172
Total set size: 9177
Total coverage sets: 4755
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.8354
Average Coverage Rate in runs 1: 0.951

Running experiment 2/10...
q_hat = 0.7021471470594406
Total set size: 9073
Total coverage sets: 4723
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.8146
Average Coverage Rate in runs 2: 0.9446

Running experiment 3/10...
q_hat = 0.71078562438488
Total set size: 9226
Total coverage sets: 4744
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.8452
Average Coverage Rate in runs 3: 0.9488

Running experiment 4/10...
q_hat = 0.7150522679090501
Total set siz

## Result
$\alpha$=0.1
- Final Average **Prediction Set Size: 1.51 / 10**
- Final Average **Coverage: 89.89%**  

$\alpha$=0.05
- Final Average **Prediction Set Size: 1.83 / 10**
- Final Average **Coverage: 94.98%**